In [ ]:
!pip install easyocr
!pip install imutils


In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr
import glob


In [ ]:
# Path to the folder containing images
image_folder = "/home/ep/Documents/Github/Computer_Vision/ESRGAN-master/results/**/*"

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Get a list of all image paths in the folder
image_paths = glob.glob(image_folder, recursive=True)

# Process each image
for image_path in image_paths:
    print(f"Processing: {image_path}")
    img = cv2.imread(image_path)

    if img is None:
        print(f"Unable to read image: {image_path}")
        continue

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply filters and edge detection
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)  # Noise reduction
    edged = cv2.Canny(bfilter, 30, 200)  # Edge detection

    # Find contours
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    # Detect the location of the contour
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:  # Looking for a quadrilateral
            location = approx
            break

    if location is None:
        print("No contour detected.")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title("Original Image (No Contour Detected)")
        plt.show()
        continue

    # Create a mask for the detected contour
    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0, 255, -1)
    new_image = cv2.bitwise_and(img, img, mask=mask)

    # Extract the region of interest
    (x, y) = np.where(mask == 255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]

    # Use EasyOCR to read text from the cropped image
    result = reader.readtext(cropped_image)

    if result:
        text = result[0][-2]
        print(f"Detected Text: {text}")

        # Annotate the image with the detected text
        font = cv2.FONT_HERSHEY_SIMPLEX
        img = cv2.putText(img, text=text, org=(approx[0][0][0], approx[1][0][1]+60), 
                          fontFace=font, fontScale=1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
        img = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0, 255, 0), 3)
    else:
        print("No text detected.")
        text = "No text detected"

    # Visualize the result
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"Processed Image: {text}")
    plt.show()
